#Preprocessing of the Data set

**Import tab separated data**

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler

**Download dataset file from https://www.kaggle.com/code/hainescity/company-s-ideal-customers** and upload it in the next step below

In [3]:
from google.colab import files

uploaded = files.upload()

Saving marketing_campaign.csv to marketing_campaign (1).csv


In [4]:
data = pd.read_csv("marketing_campaign.csv", delimiter='\t')

In [5]:
data.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


**Drop Irrelevant rows**

In [6]:
data = data.drop(
    ['ID', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds',
     'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'Complain', 'Z_CostContact',
     'Z_Revenue', 'Response'], axis=1)

In [7]:
data = data.sort_index(axis=1)

In [8]:
for column in data:
    print(column)

AcceptedCmp1
AcceptedCmp2
AcceptedCmp3
AcceptedCmp4
AcceptedCmp5
Dt_Customer
Education
Income
Kidhome
Marital_Status
NumDealsPurchases
Recency
Teenhome
Year_Birth


**Creating a subset of promo data for easier manipulation**

In [9]:
promo_data = data.iloc[:, [0, 1, 2, 3, 4]]

In [10]:
promo_data

,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
2235,0,0,0,0,0
2236,1,0,0,0,0
2237,0,0,0,1,0
2238,0,0,0,0,0


**Determining the first succesful Campaign**

In [11]:
count_row = promo_data.shape[0]
number_cmp = []

In [12]:
for i in range(count_row):
    if promo_data.iloc[i]['AcceptedCmp1'] == 1:
        number_cmp.append(1)
    elif promo_data.iloc[i]['AcceptedCmp2'] == 1:
        number_cmp.append(2)
    elif promo_data.iloc[i]['AcceptedCmp3'] == 1:
        number_cmp.append(3)
    elif promo_data.iloc[i]['AcceptedCmp4'] == 1:
        number_cmp.append(4)
    elif promo_data.iloc[i]['AcceptedCmp5'] == 1:
        number_cmp.append(5)
    else:
        number_cmp.append(0)

#print(number_cmp)

In [13]:
data['Succesful_cmp'] = number_cmp

In [14]:
data = data.drop(['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', ], axis=1)


**Cleaned data:**

In [15]:
data

,Dt_Customer,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp
0,04-09-2012,Graduation,58138.0,0,Single,3,58,0,1957,0
1,08-03-2014,Graduation,46344.0,1,Single,2,38,1,1954,0
2,21-08-2013,Graduation,71613.0,0,Together,1,26,0,1965,0
3,10-02-2014,Graduation,26646.0,1,Together,2,26,0,1984,0
4,19-01-2014,PhD,58293.0,1,Married,5,94,0,1981,0
...,...,...,...,...,...,...,...,...,...,...
2235,13-06-2013,Graduation,61223.0,0,Married,2,46,1,1967,0
2236,10-06-2014,PhD,64014.0,2,Together,7,56,1,1946,1
2237,25-01-2014,Graduation,56981.0,0,Divorced,1,91,0,1981,4
2238,24-01-2014,Master,69245.0,0,Together,2,8,1,1956,0


**Drop 'age' outliers with Year_Birth <1940**

In [16]:
data = data[data.Year_Birth >= 1940][:]

data

,Dt_Customer,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp
0,04-09-2012,Graduation,58138.0,0,Single,3,58,0,1957,0
1,08-03-2014,Graduation,46344.0,1,Single,2,38,1,1954,0
2,21-08-2013,Graduation,71613.0,0,Together,1,26,0,1965,0
3,10-02-2014,Graduation,26646.0,1,Together,2,26,0,1984,0
4,19-01-2014,PhD,58293.0,1,Married,5,94,0,1981,0
...,...,...,...,...,...,...,...,...,...,...
2235,13-06-2013,Graduation,61223.0,0,Married,2,46,1,1967,0
2236,10-06-2014,PhD,64014.0,2,Together,7,56,1,1946,1
2237,25-01-2014,Graduation,56981.0,0,Divorced,1,91,0,1981,4
2238,24-01-2014,Master,69245.0,0,Together,2,8,1,1956,0


**Convert strings to numerical value:**

**Education ->**
     Basic = 0
     2n Cycle = 1
     Graduation= 2
     Master = 3
     PhD = 4
**Marital_Status ->**
    Absurd, Alone, YOLO, Single = 0
    Divorced, Widow = 1
    Together = 2
    Married = 3

In [17]:
data['Marital_Status'] = data['Marital_Status'].replace({'Alone': 'Single', 'Absurd': 'Single', 'YOLO': 'Single'})

data['Marital_Status'] = data['Marital_Status'].replace(
    {'Single': 0, 'Divorced': '1', 'Widow': '1', 'Together': 2, 'Married': 3})
data['Education'] = data['Education'].replace({'Basic': 0, '2n Cycle': 1, 'Graduation': 2, 'Master': 3, 'PhD': 4})

data

,Dt_Customer,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp
0,04-09-2012,2,58138.0,0,0,3,58,0,1957,0
1,08-03-2014,2,46344.0,1,0,2,38,1,1954,0
2,21-08-2013,2,71613.0,0,2,1,26,0,1965,0
3,10-02-2014,2,26646.0,1,2,2,26,0,1984,0
4,19-01-2014,4,58293.0,1,3,5,94,0,1981,0
...,...,...,...,...,...,...,...,...,...,...
2235,13-06-2013,2,61223.0,0,3,2,46,1,1967,0
2236,10-06-2014,4,64014.0,2,2,7,56,1,1946,1
2237,25-01-2014,2,56981.0,0,1,1,91,0,1981,4
2238,24-01-2014,3,69245.0,0,2,2,8,1,1956,0


In [18]:
editedData = data
editedData.Dt_Customer = pd.to_datetime(editedData.Dt_Customer, dayfirst=True)


In [19]:
year, month = 0, 0

months = []

for i in range(editedData.shape[0]):
    year = editedData.iloc[i]['Dt_Customer'].year
    month = editedData.iloc[i]['Dt_Customer'].month

    if year == 2012:
        months.append(month)
    elif year == 2013:
        months.append((12 + month))
    elif year == 2014:
        months.append((24 + month))

editedData['Months_After_2012'] = months


editedData.drop('Dt_Customer',axis=1)


,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp,Months_After_2012
0,2,58138.0,0,0,3,58,0,1957,0,9
1,2,46344.0,1,0,2,38,1,1954,0,27
2,2,71613.0,0,2,1,26,0,1965,0,20
3,2,26646.0,1,2,2,26,0,1984,0,26
4,4,58293.0,1,3,5,94,0,1981,0,25
...,...,...,...,...,...,...,...,...,...,...
2235,2,61223.0,0,3,2,46,1,1967,0,18
2236,4,64014.0,2,2,7,56,1,1946,1,30
2237,2,56981.0,0,1,1,91,0,1981,4,25
2238,3,69245.0,0,2,2,8,1,1956,0,25


In [20]:
editedData = editedData.drop('Dt_Customer',axis=1)


**Remove the Outlier with "income" >600.000**

In [21]:
editedData = editedData.drop(editedData[editedData.Income >600000].index)

In [22]:
#Check if succesfully removed
num_rows = editedData.shape[0]

for i in range(num_rows):
    if editedData.iloc[i]['Income'] == 666666:
        print(i)
        print(editedData.iloc[i]['Income'])

In [23]:
from google.colab import files
editedData.to_csv("out1.csv") 
#files.download("out1.csv")

**Replace NaN Income values with mean of Income**

In [24]:
data = pd.read_csv("out1.csv",na_values="NAN");
data['Income'] = data['Income'].fillna(value=data['Income'].mean())

In [25]:
data

,Unnamed: 0,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp,Months_After_2012
0,0,2,58138.0,0,0,3,58,0,1957,0,9
1,1,2,46344.0,1,0,2,38,1,1954,0,27
2,2,2,71613.0,0,2,1,26,0,1965,0,20
3,3,2,26646.0,1,2,2,26,0,1984,0,26
4,4,4,58293.0,1,3,5,94,0,1981,0,25
...,...,...,...,...,...,...,...,...,...,...,...
2231,2235,2,61223.0,0,3,2,46,1,1967,0,18
2232,2236,4,64014.0,2,2,7,56,1,1946,1,30
2233,2237,2,56981.0,0,1,1,91,0,1981,4,25
2234,2238,3,69245.0,0,2,2,8,1,1956,0,25


In [26]:
arrIncome = np.array(data['Income'])
arrIncome = arrIncome.reshape(-1,1)

In [27]:
scaler = StandardScaler()
arrIncome = scaler.fit_transform(arrIncome)

In [28]:
arrIncome

array([[ 0.2886582 ],
       [-0.26229348],
       [ 0.91813708],
       ...,
       [ 0.23460944],
       [ 0.80751697],
       [ 0.04251911]])

In [29]:
data['Income'] = arrIncome

In [32]:
data.head()

,Unnamed: 0,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp,Months_After_2012
0,0,2,0.288658,0,0,3,58,0,1957,0,9
1,1,2,-0.262293,1,0,2,38,1,1954,0,27
2,2,2,0.918137,0,2,1,26,0,1965,0,20
3,3,2,-1.182477,1,2,2,26,0,1984,0,26
4,4,4,0.295899,1,3,5,94,0,1981,0,25


In [34]:
data = data.drop(['Unnamed: 0'], axis=1)

In [35]:
data.head()

,Education,Income,Kidhome,Marital_Status,NumDealsPurchases,Recency,Teenhome,Year_Birth,Succesful_cmp,Months_After_2012
0,2,0.288658,0,0,3,58,0,1957,0,9
1,2,-0.262293,1,0,2,38,1,1954,0,27
2,2,0.918137,0,2,1,26,0,1965,0,20
3,2,-1.182477,1,2,2,26,0,1984,0,26
4,4,0.295899,1,3,5,94,0,1981,0,25


In [36]:
from google.colab import files
data.to_csv('finalDataSet.csv')
files.download("finalDataSet.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>